In [24]:
from scrapli_netconf.driver import NetconfScrape
my_device = {
    "host": "192.168.99.91",
    "auth_username": "apiuser",
    "auth_password": "apipassword",
    "auth_strict_key": False,    
}

conn = NetconfScrape(**my_device)
conn.open()
xmlns = "http://cisco.com/ns/yang/cisco-nx-os-device"

In [25]:
# get all evpn related configs by netconf
vlan_id = 333
vxlan_id = 10333

xpath_nve = f"/System/eps-items/epId-items/Ep-list/epId=1/nws-items/vni-items/Nw-list[]/vni={vxlan_id}"
xpath_svi = f"/System/intf-items/svi-items/If-list[]/id=vlan{vlan_id}"
xpath_vlan = f"/System/bd-items/bd-items/BD-list/fabEncap=vlan-{vlan_id}"
xpath_evpn = f"/System/evpn-items/bdevi-items/BDEvi-list[]/encap=vxlan-{vxlan_id}"

rpc_nve = ypath2xml(xpath_nve, xmlns)
rpc_svi = ypath2xml(xpath_svi, xmlns)
rpc_vlan = ypath2xml(xpath_vlan, xmlns)
rpc_evpn = ypath2xml(xpath_evpn, xmlns)

response = conn.get(filter_=rpc_nve, filter_type="subtree")
print(response.result)
response = conn.get(filter_=rpc_svi, filter_type="subtree")
print(response.result)
response = conn.get(filter_=rpc_vlan, filter_type="subtree")
print(response.result)
response = conn.get(filter_=rpc_evpn, filter_type="subtree")
print(response.result)

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="101">
  <data>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
      <eps-items>
        <epId-items>
          <Ep-list>
            <epId>1</epId>
          </Ep-list>
        </epId-items>
      </eps-items>
    </System>
  </data>
</rpc-reply>

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="102">
  <data>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device"/>
  </data>
</rpc-reply>

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="103">
  <data>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device"/>
  </data>
</rpc-reply>

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="104">
  <data>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device"/>
  </data>
</rpc-reply>



In [26]:
# delete evpn
vlan_id = 333
vxlan_id = 10333

del_evpn_conf = ('<config><System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">' + 
        ypath2xml(f"/eps-items/epId-items/Ep-list/epId=1/nws-items/vni-items/Nw-list[]/vni={vxlan_id}", operation="remove") + 
        ypath2xml(f"/intf-items/svi-items/If-list[]/id=vlan{vlan_id}", operation="remove") + 
        ypath2xml(f"/bd-items/bd-items/BD-list[]/fabEncap=vlan-{vlan_id}", operation="remove") + 
        ypath2xml(f"/evpn-items/bdevi-items/BDEvi-list[]/encap=vxlan-{vxlan_id}", operation="remove") + 
"</System></config>")


ppxml(del_evpn_conf)


<?xml version="1.0" ?>
<config>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <eps-items>
            <epId-items>
                <Ep-list>
                    <epId>1</epId>
                    <nws-items>
                        <vni-items>
                            <Nw-list operation="remove">
                                <vni>10333</vni>
                            </Nw-list>
                        </vni-items>
                    </nws-items>
                </Ep-list>
            </epId-items>
        </eps-items>
        <intf-items>
            <svi-items>
                <If-list operation="remove">
                    <id>vlan333</id>
                </If-list>
            </svi-items>
        </intf-items>
        <bd-items>
            <bd-items>
                <BD-list operation="remove">
                    <fabEncap>vlan-333</fabEncap>
                </BD-list>
            </bd-items>
        </bd-items>
        <evpn-items>
          

In [27]:
response = conn.edit_config(config=del_evpn_conf, target="running")
print(response.result)

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="105">
  <ok/>
</rpc-reply>



In [28]:
conn.close()

In [13]:
xmlns = "http://cisco.com/ns/yang/cisco-nx-os-device"
xpath = "/System/intf-items/svi-items/If-list[]/id=vlan111/mtu=9000/descr=blablabla/adminSt=up/rtvrfMbr-items/tDn=//System//inst-items//Inst-list[name='Tenant-1']"

ppxml(ypath2xml(xpath, xmlns=xmlns, operation="replace"))

<?xml version="1.0" ?>
<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
    <intf-items>
        <svi-items>
            <If-list operation="replace">
                <id>vlan111</id>
                <mtu>9000</mtu>
                <descr>blablabla</descr>
                <adminSt>up</adminSt>
                <rtvrfMbr-items>
                    <tDn>/System/inst-items/Inst-list[name='Tenant-1']</tDn>
                </rtvrfMbr-items>
            </If-list>
        </svi-items>
    </intf-items>
</System>



In [2]:
from nornir import InitNornir
from nornir.core.task import Task, Result        
nr = InitNornir(config_file="config_nornir.yaml")
nr = nr.filter(site="site1")



In [4]:
vlan_id = 1234
vlan_name = "VL1234"
vrf_name = "Tenant-1"
mtu = 9000
description = "configured by netconf"
ip_address = "10.0.12.34//24"

In [5]:
from ypath import ypath_system, ppxml

vlan_get_path = f"/bd-items/bd-items/BD-list[]/fabEncap=vlan-{vlan_id}"
svi_get_path = f"/intf-items/svi-items/If-list[]/id=vlan{vlan_id}"
ipv4_get_path = f"/ipv4-items/inst-items/dom-items/Dom-list/name={vrf_name}/if-items/If-list[]/id=vlan{vlan_id}"

In [8]:

vlan_config_path = vlan_get_path + f"/name={vlan_name}/id={vlan_id}/BdState=active/adminSt=active/bridgeMode=mac/fwdCtrl=mdst-flood/fwdMode=bridge,route/mode=CE"
svi_config_path = svi_get_path + f"/mtu={mtu}/descr={description}/adminSt=up/rtvrfMbr-items/tDn=//System//inst-items//Inst-list[name='{vrf_name}']"
ipv4_config_path = ipv4_get_path + f"/addr-items/Addr-list/addr={ip_address}"

ypath_config = [vlan_config_path, svi_config_path, ipv4_config_path]

config_xml = "<config>" + ypath_system(ypath_config, operation="replace") + "</config>"
ppxml(config_xml)

def create_vxlan(task: Task, config_xml) -> Result:
    my_device = {
    "host": task.host["hostname"],
    "auth_username": task.host["username"],
    "auth_password": task.host["password"],
    "auth_strict_key": False,    
    }

    with NetconfScrape(**my_device) as conn:
        conn.open()
        resp = conn.edit_config(config=config_xml, target=datastore)
    



<?xml version="1.0" ?>
<config>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <bd-items>
            <bd-items>
                <BD-list operation="replace">
                    <fabEncap>vlan-1234</fabEncap>
                    <name>VL1234</name>
                    <id>1234</id>
                    <BdState>active</BdState>
                    <adminSt>active</adminSt>
                    <bridgeMode>mac</bridgeMode>
                    <fwdCtrl>mdst-flood</fwdCtrl>
                    <fwdMode>bridge,route</fwdMode>
                    <mode>CE</mode>
                </BD-list>
            </bd-items>
        </bd-items>
        <intf-items>
            <svi-items>
                <If-list operation="replace">
                    <id>vlan1234</id>
                    <mtu>9000</mtu>
                    <descr>configured by netconf</descr>
                    <adminSt>up</adminSt>
                    <rtvrfMbr-items>
                        <tDn>/System/in